##### 输入输出

* input() 函数暂停程序运行，同时等待键盘输入；直到回车被按下，函数的参数即为提示语，输入的类型永远是字符串型（str）
* 对 int 类型没有最大限制（相比之下， C++ 的 int 最大为 2147483647，超过这个数字会产生溢出），但是对 float 类型依然有精度限制

In [3]:
name = input('your name:')
gender = input('you are a boy?(y/n)')

welcome_str = 'Welcome to the matrix {prefix} {name}.'
welcome_dic = {
    'prefix': 'Mr.' if gender == 'y' else 'Mrs',
    'name': name
}

print('authorizing...')
print(welcome_str.format(**welcome_dic))



your name:herny
you are a boy?(y/n)
authorizing...
Welcome to the matrix Mrs herny.


In [19]:
# 简单的 NLP（自然语言处理）任务
## 可以给 read 指定参数 size ，用来表示读取的最大长度。还可以通过 readline() 函数，每次读取一行，这种做法常用于数据挖掘（Data Mining）中的数据清洗，在写一些小的程序时非常轻便
## with 语句，就不需要显式调用 close()
import re


def parse(text):
    text = re.sub(r'[^\w]', ' ', text)
    text = text.lower()
    word_list = text.split(' ')
    word_list = filter(None, word_list)

    word_cnt = {}
    for word in word_list:
        if word not in word_cnt:
            word_cnt[word] = 0
        word_cnt[word] += 1
    sorted_word_cnt = sorted(
        word_cnt.items(), key=lambda kv: kv[1], reverse=True)

    return sorted_word_cnt


with open('in.txt', 'r') as fin:
    text = fin.read()

word_and_freq = parse(text)
with open('out.txt', 'w') as fout:
    for word,freq in word_and_freq:
        fout.write('{} {}\n'.format(word, freq))

<class 'list'>


In [24]:
# JSON
import json

params = {
    'symbol': '123456',
    'type': 'limit',
    'price': 123.4,
    'amount': 23
}

params_str = json.dumps(params)

print('after json serialization')
print('type of params_str = {}, params_str = {}'.format(type(params_str), params))

original_params = json.loads(params_str)

print('after json deserialization')
print('type of original_params = {}, original_params = {}'.format(type(original_params), original_params))

after json serialization
type of params_str = <class 'str'>, params_str = {'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}
after json deserialization
type of original_params = <class 'dict'>, original_params = {'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}


##### 思考

* 制定长度，获取到下一个非字符位置，并存储
* 有限容量网盘同步大文件
    * 控制文件加状态：服务端传完，客户端才可以读取
    * 检测文件是否传完

In [25]:
import re

CHUNK_SIZE = 100 # 一次最多读取的字符长度

# 这个函数每次会接收上一次得到的 last_word，然后和这次的 text 合并起来处理。
# 合并后判断最后一个词有没有可能连续，并分离出来，然后返回。
# 这里的代码没有 if 语句，但是仍然是正确的，可以想一想为什么。
def parse_to_word_list(text, last_word, word_list):
    text = re.sub(r'[^\w ]', ' ', last_word + text)
    text = text.lower()
    cur_word_list = text.split(' ')
    cur_word_list, last_word = cur_word_list[:-1], cur_word_list[-1]
    word_list += filter(None, cur_word_list)
    
    return last_word

def solve():
    with open('in.txt', 'r') as fin:
        word_list, last_word = [], ''
        while True:
            text = fin.read(CHUNK_SIZE)
            if not text:
                break # 读取完毕，中断循环
            last_word = parse_to_word_list(text, last_word, word_list)

        word_cnt = {}
        for word in word_list:
            if word not in word_cnt:
                word_cnt[word] = 0
            word_cnt[word] += 1

        sorted_word_cnt = sorted(word_cnt.items(), key=lambda kv: kv[1], reverse=True)
        return sorted_word_cnt

print(solve())

[('and', 15), ('be', 13), ('will', 11), ('to', 11), ('the', 10), ('of', 10), ('a', 8), ('we', 8), ('day', 6), ('able', 6), ('every', 6), ('together', 6), ('i', 5), ('have', 5), ('dream', 5), ('that', 5), ('one', 5), ('with', 5), ('this', 5), ('in', 4), ('shall', 4), ('free', 4), ('when', 4), ('little', 3), ('black', 3), ('white', 3), ('made', 3), ('faith', 3), ('at', 3), ('last', 3), ('children', 2), ('nation', 2), ('by', 2), ('their', 2), ('today', 2), ('alabama', 2), ('boys', 2), ('girls', 2), ('join', 2), ('hands', 2), ('mountain', 2), ('places', 2), ('all', 2), ('it', 2), ('our', 2), ('hope', 2), ('up', 2), ('freedom', 2), ('ring', 2), ('from', 2), ('god', 2), ('men', 2), ('my', 1), ('four', 1), ('live', 1), ('where', 1), ('they', 1), ('not', 1), ('judged', 1), ('color', 1), ('skin', 1), ('but', 1), ('content', 1), ('character', 1), ('down', 1), ('its', 1), ('vicious', 1), ('racists', 1), ('right', 1), ('there', 1), ('as', 1), ('sisters', 1), ('brothers', 1), ('valley', 1), ('exalt

In [26]:
 # server.py
# 我们假设 server 电脑上的所有的文件都在 BASR_DIR 中，为了简化不考虑文件夹结构，网盘的路径在 NET_DIR

import os
from shutil import copyfile
import time

BASE_DIR = 'server/'
NET_DIR = 'net/'

def main():
    filenames = os.listdir(BASE_DIR)
    for i, filename in enumerate(filenames):
        print('copying {} into net drive... {}/{}'.format(filename, i + 1, len(filenames)))
        copyfile(BASE_DIR + filename, NET_DIR + filename)
        print('copied {} into net drive, waiting client complete... {}/{}'.format(filename, i + 1, len(filenames)))

        while os.path.exists(NET_DIR + filename):
            time.sleep(3)

    print('transferred {} into client. {}/{}'.format(filename, i + 1, len(filenames)))

if __name__ == "__main__":
main()

IndentationError: expected an indented block (<ipython-input-26-9f7caff9aba2>, line 24)

In [ ]:
# client.py
# 我们假设 client 电脑上要输出的文件夹在 BASR_DIR ，网盘的路径在 NET_DIR

import os
from shutil import copyfile
import time

BASE_DIR = 'client/'
NET_DIR = 'net/'

def main():
    while True:
        filenames = os.listdir(NET_DIR)
        for filename in filenames:
            print('downloading {} into local disk...'.format(filename))
            copyfile(NET_DIR + filename, BASE_DIR + filename)
            os.remove(NET_DIR + filename) # 我们需要删除这个文件，网盘会提我们同步这个操作，从而 server 知晓已完成
            print('downloaded {} into local disk.'.format(filename))
        time.sleep(3)

if __name__ == "__main__":
main()